## ВШЭ, ФКН, Программа "Специалист по Data Science" (2021/2022)
## Курс "Прикладные задачи анализа данных"
### ДЗ №1: Toxic Comments Classification
#### *Выполнил: Кирилл Н., 09.05.2022 г.*

__Задание:__ Обучить бинарный классификатор для поиска токсичного контента (твитов).

Описание файлов:
* train_data.csv - обучающая выборка
* test_data.csv - тестовая выборка
* sample_submission.csv - пример файла с решением (с dummy предсказаниями)

[Ссылка на Kaggle](https://www.kaggle.com/competitions/toxic-comments-classification-apdl-2022)

__Содержание:__

1. [BoW / TF-IDF + Linear model](#bow)
2. [Модель на векторах слов w2v](#w2v)
3. [Supervised FastText](#fasttext)

In [1]:
from pathlib import Path
import sys

try:
    import google.colab
    IN_COLAB = True
    
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    google.colab.drive.mount("/content/gdrive")
    ROOT = Path("/content/gdrive/My Drive/")
    assert ROOT.is_dir(), "Wrong path"
    sys.path.append(str(ROOT))
    
else:
    ROOT = Path(".")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

import numpy as np

In [3]:
train_data = pd.read_csv(f'{ROOT}/train_data.csv')

In [4]:
train_data.sample(5)

,comment,toxic
9397,"Уж лучше пусть смотрят нашу пропаганду, чем пиндоскую.\n",1.0
3766,"Сначала запретить въезд чуркам и выдворить уже приезжих, а потом уже уидем на сколько упадёт преступность в стране.\n",1.0
1936,"потрясающе! очень красиво, луна прям светится)\n",0.0
7323,"Я сама, как услышала в первый раз их разговор, обалдела.Думала, они понимающие. А оказывается меня уважать не за что .\n",0.0
7952,"ну ты тупой, блядь, совет выкрутить к тебе исходил из предположения, что у тебя наушники ашановские, которое ты, кстати, и не опровергнул. я и сам его не слушаю на 200 громкости, но всю эту хуйню слышно и без этого\n",1.0


In [5]:
test_data = pd.read_csv(f'{ROOT}/test_data.csv', index_col=0)

In [6]:
test_data.sample(5)

,comment
comment_id,
3127,"Хуита что-то какая-то. Ни сисек, ни жопы, рязанское ебало с прыщами, да ещё и блондинка. Уноси.\n"
2888,"Анимация травы очень прикольно вышла А вообще это красиво, пиксельная красота, жду новых комикс скетчей, и других картинок)\n"
3393,"SAMP 2008 года - это лучшее, во что можно было играть по сети. Сейчас там настолько токсичный и школьный контингент, что ничего общего с ним иметь не хочется.\n"
2444,"мимо хикка Ты не хикка, ты чмо с отклонениями в психике. Поссал на тебя.\n"
1594,"Попробуйте вискозные полотенца ака тряпки для уборки. Одноразовые такие. Они еще круче зевы, нежные, как попка пэрсыка\n"


In [7]:
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
!pip install pymorphy2
!pip install pymorphy2-dicts
!pip install DAWG-Python

     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 11.0 MB/s 
     |████████████████████████████████| 7.1 MB 4.6 MB/s 


In [35]:
import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from tqdm import tqdm
from multiprocessing import Pool

import nltk
import nltk.data
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, RegexpTokenizer
nltk.download('punkt')
nltk.download('stopwords')

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1. BoW / TF-IDF + Linear model <a id="bow"></a>

- Провести препроцессинг: подумайте о регистре, стоп-словах, нормализации, очистке от небуквенных символов.
- Получить для каждого объекта выборки список токенов.
- Для списка токенов (на train выборке!) посчитать модель BoW / tf-idf.
- Обучить на полученном представлении линейную модель (например, логистическую регрессию).

In [9]:
m = MorphAnalyzer()
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]

mystopwords = stopwords.words('russian') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

regex = re.compile("[А-Яа-яA-z]+")
def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [10]:
with Pool(8) as p:
    lemmas_train = list(tqdm(map(clean_text, train_data['comment']), total=len(train_data)))
    
train_data['lemmas'] = lemmas_train

  0%|          | 0/10809 [00:00<?, ?it/s]

In [11]:
train_data.sample(5)

,comment,toxic,lemmas
10676,"Не нужно теперь будет ждать когда бабка откинется, чтобы не мешаться под ногами в квартире.",1.0,нужно ждать бабка откинуться мешаться нога квартира
7675,Работает - это когда рецидивов больше нет\n,0.0,работать рецидив большой
1375,"Фанера 4мм, подача 50 мм с, 2400 об мин\n",0.0,фанера подача мина
5561,"Европы и США Это ещё ладно, но ещё больше трудятся во всяких чуркистанах.\n",1.0,европа ладный большой трудиться всякий чуркистан
7456,"Я бы нашел ту тварь, написал бы ей деликатно что со стороны юриспруденции она совершила критическую ошибку.. и доебывал ее до тех пор пока она сука не взвыла, методов много..\n",1.0,найти тварь написать деликатно сторона юриспруденция совершить критический ошибка доебывать пора пока сука взвыть метод


In [12]:
with Pool(8) as p:
    lemmas_test = list(tqdm(map(clean_text, test_data['comment']), total=len(test_data))) 

test_data['lemmas'] = lemmas_test

  0%|          | 0/3603 [00:00<?, ?it/s]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_data.lemmas, train_data.toxic)
X_test = test_data.lemmas

In [ ]:
vec = CountVectorizer(ngram_range=(1, 2)) # строим BoW для слов
bow = vec.fit_transform(X_train)

In [ ]:
clf_lr = LogisticRegression(random_state=42, max_iter=500)
clf_lr.fit(bow, y_train)

y_val_pred = clf_lr.predict(vec.transform(X_val))
print(classification_report(y_val_pred, y_val))

              precision    recall  f1-score   support

         0.0       0.96      0.85      0.90      2062
         1.0       0.65      0.87      0.74       641

    accuracy                           0.86      2703
   macro avg       0.80      0.86      0.82      2703
weighted avg       0.88      0.86      0.86      2703



In [ ]:
accuracy_score(y_val_pred, y_val)

0.8575656677765445

In [ ]:
y_result = clf_lr.predict(vec.transform(X_test))

solution_1 = [[i, y_result[i]] for i in range(len(X_test))]
solution_1 = pd.DataFrame(solution_1, columns=['comment_id', 'toxic'])
solution_1.to_csv('solution_1.csv', index=False)

## 2. Модель на векторах слов w2v <a id="w2v"></a>

- Провести препроцессинг: подумайте о регистре, стоп-словах, нормализации, очистке от небуквенных символов.
- Преобразовать текст в следующий формат: текст - это список предложений, каждое предложение - список слов, например, [[это, первое, предложение], [а, это, второе]].
- Обучите модель векторов слов. Если вы обучаете w2v, то на вход можно подать данные в формате п. 2. Если вы хотите обучить fastText, надо записать тексты в файл в формате "1 строка - 1 текст, слова разделены пробелом".
- Напишите функцию, которая принимает на вход текст, а возвращает средний эмбеддинг (средний по словам):

In [ ]:
'''
model = w2v()

def mean_text_embedding(sentence, embeddings_model): 
  tokens = sentence.split()
  embedings = [model(token) for token in tokens] 
  embeding_text = mean(embedings)

return embeding_text
'''

'\nmodel = w2v()\n\ndef mean_text_embedding(sentence, embeddings_model): \n  tokens = sentence.split() \n  embedings = [model(token) for token in tokens] \n  embeding_text = mean(embedings)\n\nreturn embeding_text\n'

Если размерность эмбеддинга слова у вас равна N, то и эмбеддинг текста будет иметь размерность N.

- Вычислите средний эмбеддинг для всех документов вашей коллекции.
- Обучите на полученных векторах модель (каждый объект выборки у вас представлен N признаками).

In [20]:
import gensim
import logging
from gensim.models import word2vec

In [21]:
# Препроцессинг

m = MorphAnalyzer()
mystopwords = stopwords.words("russian")
def sent_to_wordlist(sentence, pymorphy=m, remove_stopwords=True, stopwords=mystopwords):
    sentence = re.sub("[^А-Яа-яA-z]"," ", sentence)
    words = sentence.lower().split()
    words = [pymorphy.parse(w)[0].normal_form for w in words]
    if remove_stopwords:
        words = [w for w in words if not w in stopwords]
    return words

tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')
def text_to_sentences(text, tokenizer=tokenizer, pymorphy=m, remove_stopwords=True):
    raw_sentences = tokenizer.tokenize(text.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(sent_to_wordlist(raw_sentence))
    return sentences

In [22]:
with Pool(8) as p:
    sentences_train = list(tqdm(p.imap(text_to_sentences, train_data["comment"]), total=len(train_data)))

train_data['sentences'] = sentences_train

  0%|          | 0/10809 [00:00<?, ?it/s]

In [19]:
train_data.sample(5)

,comment,toxic,lemmas
4049,"Да жидяра это, что тут думать\n",1.0,жидярый думать
632,"Достаточно сделать таким образом, что и ремонтники и операторы станка - его собственники. И з п они будут получать согласно выпущенных изделий. Тогда у них мотивация держать станок в рабочем состоянии и выпускать максимум.\n",0.0,достаточно сделать образ ремонтник оператор станок собственник получать согласно выпустить изделие мотивация держать станок рабочий состояние выпускать максимум
3087,"Лично я придерживаюсь мнения, что это красивый элемент декора, никаких отсылок к магическим свойствам не делаю. Но это совершенно не отменяет того, что некоторые люди верят в свойства ловца снов и он работает - согласно древнеиндейским обычаям этот амулет защищает сны спящего от кошмаров, пропуская лишь добрые и хорошие сновидения. Как-то так вкратце.\n",0.0,лично придерживаться мнение красивый элемент декор никакой отсылка магический свойство делать совершенно отменять некоторый человек верить свойство ловец работать согласно древнеиндейский обычай амулет защищать спать кошмар пропускать лишь добрый хороший сновидение вкратце
4116,А в твоем блять не стояли! вы же пердуны старый все высоко духовные интеллектуалы.\n,1.0,твой блядь стоять пердун старый высоко духовный интеллектуал
2643,"Хуйня этот ваш дэбиль. Я думал, он всех будет хуесосить и избивать направо и налево.\n",1.0,хуйня дэбиль думать весь хуесосить избивать направо налево


In [23]:
with Pool(8) as p:
    sentences_test = list(tqdm(p.imap(text_to_sentences, test_data["comment"]), total=len(test_data)))

test_data['sentences'] = sentences_test

  0%|          | 0/3603 [00:00<?, ?it/s]

**Обучение собственной модели**

In [24]:
flat_sentences = [item for sublist in sentences_train for item in sublist]

In [25]:
print("Training my model...")

%time my_model_ru = word2vec.Word2Vec(flat_sentences, workers=4, size=300, min_count=10, window=10, sample=1e-3)

Training my model...
CPU times: user 4.06 s, sys: 73.2 ms, total: 4.14 s
Wall time: 2.72 s


In [ ]:
print(len(my_model_ru.wv.vocab))

3157


In [26]:
X_train, X_val, y_train, y_val = train_test_split(train_data.sentences, train_data.toxic)
X_test = test_data.sentences

In [27]:
def mean_text_embedding(text, model=my_model_ru):
    flat_text = [item for sublist in text for item in sublist]
    text_embeddings = [np.array(model[token]) for token in flat_text if token in model]
    
    if len(text_embeddings) == 0:
        mean_text_emb = np.zeros(300)  
    else:
        mean_text_emb = np.mean(np.array(text_embeddings), axis=0)

    return mean_text_emb

In [28]:
X_train_emb = [mean_text_embedding(text, my_model_ru) for text in X_train]
X_val_emb = [mean_text_embedding(text, my_model_ru) for text in X_val]
X_test_emb = [mean_text_embedding(text, my_model_ru) for text in X_test]

In [33]:
clf_lr = LogisticRegression()
clf_lr.fit(X_train_emb, y_train)
y_val_pred = clf_lr.predict(X_val_emb)

print(classification_report(y_val_pred, y_val))

              precision    recall  f1-score   support

         0.0       0.98      0.67      0.80      2590
         1.0       0.08      0.68      0.15       113

    accuracy                           0.67      2703
   macro avg       0.53      0.68      0.47      2703
weighted avg       0.94      0.67      0.77      2703



In [34]:
accuracy_score(y_val_pred, y_val)

0.6699963004069552

In [ ]:
y_result = clf_lr.predict(X_test_emb)

solution_2 = [[i, y_result[i]] for i in range(len(test_data))]
solution_2 = pd.DataFrame(solution_2, columns=['comment_id', 'toxic'])
solution_2.to_csv('solution_2.csv', index=False)

**Использование готовой модели**

In [ ]:
!wget https://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz

--2022-05-09 07:31:53--  https://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 317128925 (302M) [application/x-gzip]
Saving to: ‘ruscorpora_mystem_cbow_300_2_2015.bin.gz’

ruscorpora_mystem_c 100%[===================>] 302.44M  25.8MB/s    in 12s     

2022-05-09 07:32:06 (24.5 MB/s) - ‘ruscorpora_mystem_cbow_300_2_2015.bin.gz’ saved [317128925/317128925]



In [ ]:
model_path = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'
model_ru2 = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_data.sentences, train_data.toxic)
X_test = test_data.sentences

In [ ]:
X_train_emb = [mean_text_embedding(text, model_ru2) for text in X_train]
X_val_emb = [mean_text_embedding(text, model_ru2) for text in X_val]
X_test_emb = [mean_text_embedding(text, model_ru2) for text in X_test]

In [ ]:
clf_lr = LogisticRegression()
clf_lr.fit(X_train_emb, y_train)
y_val_pred = clf_lr.predict(X_val_emb)

print(classification_report(y_val_pred, y_val))

              precision    recall  f1-score   support

         0.0       1.00      0.66      0.80      2703
         1.0       0.00      0.00      0.00         0

    accuracy                           0.66      2703
   macro avg       0.50      0.33      0.40      2703
weighted avg       1.00      0.66      0.80      2703



In [ ]:
accuracy_score(y_val_pred, y_val)

0.6722160562338143

## 3. Supervised FastText <a id="fasttext"></a>

In [ ]:
!pip install fasttext

     |████████████████████████████████| 68 kB 3.2 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3140310 sha256=0058427602225a82822acd19f56ce0515bcffbf9efb863b06247ee1bdcd8d550
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz

--2022-05-09 07:41:51--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4496459151 (4.2G) [application/octet-stream]
Saving to: ‘cc.ru.300.bin.gz’

cc.ru.300.bin.gz    100%[===================>]   4.19G  37.4MB/s    in 90s     

2022-05-09 07:43:22 (47.6 MB/s) - ‘cc.ru.300.bin.gz’ saved [4496459151/4496459151]



In [ ]:
!gunzip cc.ru.300.bin.gz

In [13]:
import fasttext, fasttext.util

#fasttext.util.download_model('ru', if_exists='ignore')
ft = fasttext.load_model('cc.ru.300.bin')

In [14]:
X_train, X_val, y_train, y_val = train_test_split(train_data.lemmas, train_data.toxic)

In [15]:
with open('train_ft.txt', 'w') as f:
    for pair in list(zip(X_train, y_train)):
        text, label = pair
        f.write(f'__label__{label} {text.lower()}\n')

In [16]:
with open('val_ft.txt', 'w') as f:
    for pair in list(zip(X_val, y_val)):
        text, label = pair
        f.write(f'__label__{label} {text.lower()}\n')

In [17]:
clf_ft = fasttext.train_supervised('train_ft.txt')#, 'model')
y_val_pred = clf_ft.test('val_ft.txt')

print('P@1:', y_val_pred[1])#.precision)
print('R@1:', y_val_pred[2])#.recall)
print('Number of examples:', y_val_pred[0])#.nexamples)

P@1: 0.8564557898631151
R@1: 0.8564557898631151
Number of examples: 2703


In [18]:
y_result = clf_ft.predict(list(test_data['lemmas']))[0]
y_result = [label[0].split('__')[-1] for label in y_result]

solution_3 = [[i, y_result[i]] for i in range(len(test_data))]
solution_3 = pd.DataFrame(solution_3, columns=['comment_id', 'toxic'])
solution_3.to_csv('solution_3.csv', index=False)